In [19]:
# Question 1 Local Machine
import torch

if torch.cuda.is_available():
    print("GPU is available.")
    print("GPU device name:", torch.cuda.get_device_name(0))
    print("Number of GPUs:", torch.cuda.device_count())
else:
    print("GPU is not available; using CPU.")

GPU is not available; using CPU.


# Question 1 result from google collab

```
import torch

if torch.cuda.is_available():
    print("GPU is available.")
    print("GPU device name:", torch.cuda.get_device_name(0))
    print("Number of GPUs:", torch.cuda.device_count())
else:
    print("GPU is not available; using CPU.")
    
```

GPU is available.
GPU device name: NVIDIA A100-SXM4-80GB
Number of GPUs: 1

In [20]:
# Question 2, Use Calculus and PyTorch to Derive the Gradient Descent


import torch
import math
import sympy as sp


def loss_math(x, y):
    try:
        loss = x / (y + x**2) + math.exp(-y * x)
        return loss
    except ZeroDivisionError:
        return float('inf')

def loss(x, y):
    try:
        return x / (y + x**2) + torch.exp(-y * x)
    except (ZeroDivisionError, RuntimeError):
        return torch.tensor(float('inf'))

def dloss_dx(x, y):
    try:
        return (y + x**2 - 2*x**2) / (y + x**2)**2 - y * math.exp(-y * x)
    except ZeroDivisionError:
        return float('inf')

def dloss_dy(x, y):
    try:
        return -x / (y + x**2)**2 + x * math.exp(-y * x)
    except ZeroDivisionError:
        return float('inf')

if __name__ == "__main__":
    # print("Loss at (0,1):", loss(0, 1))
    print("dL/dx at (0,1) with plane calculus:", dloss_dx(0, 1))
    print("dL/dy at (0,1) with plane calculus:", dloss_dy(0, 1))
    # # print("\nLoss at (1,-1):", loss(1, -1))
    # print("dL/dx at (1,-1) with plane calculus:", dloss_dx(1, -1) or 0) if error print error statement
    print("dL/dx at (1,-1) with plane calculus:", dloss_dx(1, -1))
    print("dL/dy at (1,-1) with plane calculus:", dloss_dy(1, -1))

    print("\nUsing PyTorch autograd:")

    x1 = torch.tensor(0.0, requires_grad=True)
    y1 = torch.tensor(1.0, requires_grad=True)

    x2 = torch.tensor(1.0, requires_grad=True)
    y2 = torch.tensor(-1.0, requires_grad=True)

    l1 = loss(x1, y1)
    l2 = loss(x2, y2)
    l1.backward()
    l2.backward()

    print("Grad at (0,1):")
    print("dl/dx =", x1.grad.item())
    print("dl/dy =", y1.grad.item())
    print("\nGrad at (1,-1):")
    print("dl/dx =", x2.grad.item())
    print("dl/dy =", y2.grad.item())

dL/dx at (0,1) with plane calculus: 0.0
dL/dy at (0,1) with plane calculus: 0.0
dL/dx at (1,-1) with plane calculus: inf
dL/dy at (1,-1) with plane calculus: inf

Using PyTorch autograd:
Grad at (0,1):
dl/dx = 0.0
dl/dy = -0.0

Grad at (1,-1):
dl/dx = nan
dl/dy = -inf


In [ ]:
# Question 3

%matplotlib inline
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
torch.set_printoptions(edgeitems=2, linewidth=75)


t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0,
                    8.0, 3.0, -4.0, 6.0, 13.0, 21.0])
t_f = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
t_fn = 0.1 * t_f 

add_size = 100 - t_c.size(0)


t_c_added = torch.linspace(-25, 45, steps=add_size)
t_f_added = (9.0 / 5.0) * t_c_added + 32.0

t_c = torch.cat((t_c, t_c_added), 0)
t_f = torch.cat((t_f, t_f_added), 0)

def add_gaussian_noise(t, mean=0.0, std=3.0):
    noise = torch.randn(t.size()) * std + mean
    return t + noise
t_f_noisy = add_gaussian_noise(t_f)
t_c_noisy = add_gaussian_noise(t_c)


t_f_noisy = t_f_noisy.numpy()
t_c_noisy = t_c_noisy.numpy()

t_f_train, t_f_val, t_c_train, t_c_val = train_test_split(
    t_f_noisy, t_c_noisy, test_size=0.7, random_state=42)

# back to tensors after splitting

t_f_train = torch.tensor(t_f_train, dtype=torch.float32)
t_c_train = torch.tensor(t_c_train, dtype=torch.float32)
t_f_val = torch.tensor(t_f_val, dtype=torch.float32)
t_c_val = torch.tensor(t_c_val, dtype=torch.float32)

def model(t_f, w, b): 
    return w * t_f + b

def loss_fn(t_p, t_c): # also known as criterion
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

# batch_size = 16
# batch 

w = torch.tensor(1.0, device='cpu')
b = torch.tensor(0.0, device='cpu')

train_dataset = TensorDataset(t_f_train, t_c_train)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)


def train_step(w,b, lr=0.1, epochs=200):
    w.requires_grad_(True)
    b.requires_grad_(True)

    optim = torch.optim.Adam([w, b], lr=lr)
    for epoch in range(epochs):
        total_loss = 0
        for t_f_batch, t_c_batch in train_loader:
            t_f_batch = t_f_batch.to(device='cpu')
            t_c_batch = t_c_batch.to(device='cpu')


            t_p = model(t_f_batch, w, b) # predicted farenhite temperature
            loss = loss_fn(t_p, t_c_batch) # compute the loss for this batch

            optim.zero_grad()
            loss.backward() # backwards pass to compute gradients
            optim.step() # update weights and bias
            
            total_loss += loss.item() # accumulate loss over batches
        if (epoch + 1) % 10 == 0: 
            mean_loss = total_loss / len(train_loader)
            print(f"Epoch {epoch+1}/{epochs}: w={w.item():.4f}, b={b.item():.4f}, Loss: {mean_loss:.4f}")
    return w, b

w, b = train_step(w, b)

def evaluate(t_f, t_c, w, b):
    t_p = model(t_f, w, b)
    loss = loss_fn(t_p, t_c)
    return loss.item()



print(f"\nFinal parameters: w={w.item():.4f}, b={b.item():.4f}")



SyntaxError: invalid syntax (3127789814.py, line 71)